In [ ]:
import numpy as np

import sys
sys.path.append("..")
import csaf
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

## Construct the F16 Scenario

In [ ]:
# describe initial states (via bounds)
bounds = [{'name': 'x_pos', 'type': 'continuous', 'domain': (-10000,10000)},
          {'name': 'y_pos', 'type': 'continuous', 'domain': (-10000,10000)},
          {'name': 'theta', 'type': 'continuous', 'domain': (-np.pi,np.pi)},
          {'name': 'speed', 'type': 'continuous', 'domain': (-200,200)}]

# additional constraints to reduce the initial states
constraints = [
    # keep intruder initial position at least 7000 ft away
    {'name': 'constr_1', 'constraint': '-((x[:, 1]**2 + x[:, 0]**2) - 7000**2)'},
    # keep intruder "pointed at" ownship, plus/minus 90 degrees
    # TODO: this doesn't look at all quadrants - debug?
    #{'name': 'constr_2', 'constraint': 'np.abs((np.pi + x[:, 2]) - np.arctan2(x[:, 1], x[:, 0])) - np.pi/2'}
]

def intruder_airspeed(t):
    return min(550.0 + t * 10.0, 800.0)

# Create scenario
s = csaf.test.ScenarioAcas(bounds, constraints, intruder_airspeed)

# Create optimizer
o = csaf.test.GPOptimizer()

# Create falsification goal
g = csaf.test.FalsifyAcas(optimizer=o, scenario=s)

## Solve the Optimization Problem

In [ ]:
# Run optimization with default values
g.falsify()

## Visualize the Optimal Configuration

In [ ]:
# TODO: add prettier accessors
g.optimizer.bo.plot_convergence()

In [ ]:
# print optimal configuration and objective value
print(g.optimizer.bo.x_opt)
print(g.optimizer.bo.fx_opt)

In [ ]:
from f16lib.acas import AcasScenarioViewer
scen, sys = s.get_system_under_test(g.optimizer.bo.x_opt) # relative airspeed
trajs, p = sys.simulate_tspan((0.0, 30.0), return_passed=True, terminating_conditions_all=g.properties_func)
viewer = AcasScenarioViewer(trajs, scen)
fig, ax = viewer.summary_plot()
plt.show()

In [ ]:
idxs = np.argsort(g.optimizer.bo.Y.flatten())
for idx in idxs[:10]:
    scen, sys = s.get_system_under_test(g.optimizer.bo.X[idx]) # relative airspeed
    trajs, p = sys.simulate_tspan((0.0, 30.0), return_passed=True, terminating_conditions_all=g.properties_func)
    viewer = AcasScenarioViewer(trajs, scen)
    fig, ax = viewer.summary_plot()
    plt.show()

In [ ]:
from IPython.display import HTML
from matplotlib import animation
anim = viewer.summary_video()
#writervideo = animation.FFMpegWriter(fps=30) 
#anim.save("bopt_example.mp4", writer=writervideo)
HTML(anim.to_jshtml())

In [ ]:
istates = np.array(trajs["intruder_plant"].states)
itimes = np.array(trajs["intruder_plant"].times)

plt.figure(figsize=(8, 6))
plt.plot(itimes, istates[:, 0], zorder=200, label='Airspeed')
plt.plot(itimes, [intruder_airspeed(t) for t in itimes], '--k', label='Setpoint')
plt.grid()
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Airspeed (ft/s)")